In [2]:
import pandas as pd
from scipy.io import arff

import plotly
plotly.tools.set_credentials_file(username='artemii-yanushevskyi', api_key='aRmQfG7U4SAlhISYVym7')

import plotly.plotly as py
import plotly.graph_objs as go

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
from IPython.display import Image
import plotly.io as pio

import numpy as np
import seaborn as sns

from scipy.stats.stats import pearsonr
import matplotlib.pyplot as plt
colormap = plt.cm.RdBu

exporting = False

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import OneHotEncoder

from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus

from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [3]:
data, meta = arff.loadarff('cworkTrain.arff')
df = pd.DataFrame(data)

# convert attribute values with type "object" to regular strings
objects_df = df.select_dtypes([object]) # select only atributes of object type
stack_df = objects_df.stack() # means create one column
decoded_stack_df = stack_df.str.decode('utf-8') # decode the values in the column
decoded_objects_df = decoded_stack_df.unstack() # separate into columns

# replace in df
for col in decoded_objects_df.columns:
    df[col] = decoded_objects_df[col]

# replace month with a value
df['month'] = pd.to_datetime(df.month, format='%b').dt.month

display(
    'The dataset contains {} entries and {} attributes.'.format(*df.shape),
    df.head(),
    'There are 9 categorical and 8 numerical attributes.',
    df.select_dtypes([object]).describe(),    
    df.select_dtypes([float, int]).describe().round(1)
)

'The dataset contains 36169 entries and 17 attributes.'

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,termDeposit
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,5,261.0,1.0,-1.0,0.0,unknown,no
1,44.0,technician,single,secondary,no,29.0,yes,no,unknown,5.0,5,151.0,1.0,-1.0,0.0,unknown,no
2,33.0,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,5,76.0,1.0,-1.0,0.0,unknown,no
3,47.0,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,5,92.0,1.0,-1.0,0.0,unknown,no
4,35.0,management,married,tertiary,no,231.0,yes,no,unknown,5.0,5,139.0,1.0,-1.0,0.0,unknown,no


'There are 9 categorical and 8 numerical attributes.'

,job,marital,education,default,housing,loan,contact,poutcome,termDeposit
count,36169,36169,36169,36169,36169,36169,36169,36169,36169
unique,12,3,4,2,2,2,3,4,2
top,blue-collar,married,secondary,no,yes,no,cellular,unknown,no
freq,7808,21746,18594,35512,20049,30363,23416,29621,31981


,age,balance,day,month,duration,campaign,pdays,previous
count,36169.0,36169.0,36169.0,36169.0,36169.0,36169.0,36169.0,36169.0
mean,40.9,1352.8,15.8,6.1,256.7,2.8,39.9,0.6
std,10.6,3028.9,8.3,2.4,255.6,3.1,99.8,2.4
min,18.0,-6847.0,1.0,1.0,0.0,1.0,-1.0,0.0
25%,33.0,70.0,8.0,5.0,103.0,1.0,-1.0,0.0
50%,39.0,445.0,16.0,6.0,180.0,2.0,-1.0,0.0
75%,48.0,1417.0,21.0,8.0,317.0,3.0,-1.0,0.0
max,95.0,98417.0,31.0,12.0,4918.0,63.0,871.0,275.0


In [4]:
df['termDeposit'] = df['termDeposit'].apply(lambda x: 0 if x == 'no' else 1)
df_features = df.drop(columns='termDeposit')
df_categorical = df_features.select_dtypes([object])
df_numerical = df_features.select_dtypes([int, float])

In [5]:
# create dummy variables for df_categorical
from sklearn.preprocessing import OneHotEncoder

df_one_hot = pd.concat([df_numerical, df['termDeposit']], axis=1)
catergorical_attributes = dict()
for atr in df_categorical.columns:
    df_dummies = pd.get_dummies(df[atr], prefix = atr)
    catergorical_attributes[atr] = [col.split('_')[1] for col in df_dummies.columns]
    df_one_hot = pd.concat([df_one_hot, df_dummies], axis=1) # the dataset ready to appy decision tree algorithm

display(df_one_hot.head())

,age,balance,day,month,duration,campaign,pdays,previous,termDeposit,job_admin.,...,housing_yes,loan_no,loan_yes,contact_cellular,contact_telephone,contact_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58.0,2143.0,5.0,5,261.0,1.0,-1.0,0.0,0,0,...,1,1,0,0,0,1,0,0,0,1
1,44.0,29.0,5.0,5,151.0,1.0,-1.0,0.0,0,0,...,1,1,0,0,0,1,0,0,0,1
2,33.0,2.0,5.0,5,76.0,1.0,-1.0,0.0,0,0,...,1,0,1,0,0,1,0,0,0,1
3,47.0,1506.0,5.0,5,92.0,1.0,-1.0,0.0,0,0,...,1,1,0,0,0,1,0,0,0,1
4,35.0,231.0,5.0,5,139.0,1.0,-1.0,0.0,0,0,...,1,1,0,0,0,1,0,0,0,1


In [6]:
# one hot encoded attributes place in place of the original,
# so that the order is preserved

attribute_order_one_hot = []
for i in range(len(df.columns)):
    if df.columns[i] in catergorical_attributes.keys():
        print(i, df.columns[i])
        categories = [df.columns[i] + '_' + cat for cat in catergorical_attributes[df.columns[i]]]
        attribute_order_one_hot.extend(categories)
    else:
        attribute_order_one_hot.append(df.columns[i])
        
df_one_hot_ordered = df_one_hot[attribute_order_one_hot]
df_one_hot_ordered.head()

1 job
2 marital
3 education
4 default
6 housing
7 loan
8 contact
15 poutcome


,age,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,...,month,duration,campaign,pdays,previous,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,termDeposit
0,58.0,0,0,0,0,1,0,0,0,0,...,5,261.0,1.0,-1.0,0.0,0,0,0,1,0
1,44.0,0,0,0,0,0,0,0,0,0,...,5,151.0,1.0,-1.0,0.0,0,0,0,1,0
2,33.0,0,0,1,0,0,0,0,0,0,...,5,76.0,1.0,-1.0,0.0,0,0,0,1,0
3,47.0,0,1,0,0,0,0,0,0,0,...,5,92.0,1.0,-1.0,0.0,0,0,0,1,0
4,35.0,0,0,0,0,1,0,0,0,0,...,5,139.0,1.0,-1.0,0.0,0,0,0,1,0


In [45]:
df_shuffle = shuffle(df_one_hot_ordered)
tree = DecisionTreeClassifier(min_impurity_decrease=0.01)
scores_tree = cross_val_score(tree, df_shuffle.drop('termDeposit', axis=1), df_shuffle['termDeposit'], cv=10)
scores_tree

array([0.88999447, 0.88913464, 0.88885817, 0.88802875, 0.88830523,
       0.88664639, 0.89079348, 0.88747581, 0.89021018, 0.8949115 ])

In [46]:
X_train, X_test, y_train, y_test = train_test_split(df_shuffle.drop('termDeposit', axis=1), df_shuffle['termDeposit'], test_size=0.33, random_state=45)
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.01, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [47]:
y_pred_proba = tree.predict_proba(X_test)[:, 1]
confusion_matrix(y_test, y_pred_proba > 0.5)
# 0.5 TP 199 TN 10395 FP 137 FN 1205

array([[10464,   125],
       [ 1160,   187]])

In [48]:
for t in np.linspace(0, 1, 101):
    y_pred = y_pred_proba > t # the same as tree.predict
    print(
        t,
        'TP %d' % sum((y_pred == 1) & (y_test == 1)),
        'TN %d' % sum((y_pred == 0) & (y_test == 0)),
        'FP %d' % sum((y_pred == 1) & (y_test == 0)),
        'FN %d' % sum((y_pred == 0) & (y_test == 1)),
        '\t\t\tcost', sum((y_pred == 1) & (y_test == 0)) + 10 * sum((y_pred == 0) & (y_test == 1))
    )

0.0 TP 1347 TN 0 FP 10589 FN 0 			cost 10589
0.01 TP 1347 TN 0 FP 10589 FN 0 			cost 10589
0.02 TP 1347 TN 0 FP 10589 FN 0 			cost 10589
0.03 TP 1347 TN 0 FP 10589 FN 0 			cost 10589
0.04 TP 1347 TN 0 FP 10589 FN 0 			cost 10589
0.05 TP 1347 TN 0 FP 10589 FN 0 			cost 10589
0.06 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.07 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.08 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.09 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.1 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.11 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.12 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.13 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.14 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.15 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.16 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.17 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.18 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.19 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.2 TP 819 TN 9465 FP 1124 FN 528 			cost 6404
0.21 TP

In [23]:
~(res < 0.5)

array([False, False, False, ..., False, False, False])